In [1]:
import csv
import math

import numpy as np
import tensorflow as tf
import pandas as pd
from PIL import Image
from tensorflow.keras.models import Sequential 
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, Callback
from tensorflow.keras.layers import MaxPooling2D, Conv2D, BatchNormalization, Activation, Flatten, Dense, Dropout
from tensorflow.keras.utils import Sequence
from tensorflow.keras.optimizers import Adam
from keras.constraints import maxnorm
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.backend import epsilon
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
# 0.35, 0.5, 0.75, 1.0
ALPHA = 1.0

IMAGE_SIZE = 224

EPOCHS = 30 # 200
BATCH_SIZE = 32 # 8
PATIENCE = 15

MULTI_PROCESSING = False
THREADS = 1

TRAIN_CSV = "train.csv"
VALIDATION_CSV = "validation.csv"

In [9]:
def data_generator(csv_file):

    paths = []
    classes = []

    with open(csv_file, "r") as file:
        file.seek(0)

        reader = csv.reader(file, delimiter=",")

        for index, row in enumerate(reader):
            path, _, _, _, _, _, _, _, class_ = row
            paths.append(path)
            classes.append(int(class_))

    classes_out = np.zeros((len(classes), 1), dtype=np.int8)
    images = np.zeros((len(paths), IMAGE_SIZE, IMAGE_SIZE, 3), dtype=np.float32)
    for i, f in enumerate(paths):
        img = Image.open(f)
        img = img.resize((IMAGE_SIZE, IMAGE_SIZE))
        img = img.convert('RGB')

        images[i] = np.array(img, dtype=np.float32)
        img.close()
        
        classes_out[i][0] = classes[i]

    return images, classes_out

def create_model(input_shape):
    model = Sequential()

    # Добавляем слои
    model.add(Conv2D(20, (5, 5), input_shape=input_shape, activation="relu", kernel_constraint=maxnorm(3,), padding="same"))
    model.add(MaxPooling2D(padding="same"))
    model.add(Conv2D(15, (5, 5), activation="relu", kernel_constraint=maxnorm(3,), padding="same"))
    model.add(Conv2D(8, (4, 4), activation="relu", kernel_constraint=maxnorm(3,), padding="same"))
    model.add(MaxPooling2D(padding="same"))
    model.add(Conv2D(18, (6, 6), activation="relu", kernel_constraint=maxnorm(3,), padding="same"))
    model.add(MaxPooling2D(pool_size=(2, 2), padding="same"))

    model.add(Flatten())
    model.add(Dense(512, activation="relu"))
    model.add(Dropout(0.2))
    model.add(Dense(256, activation="relu"))
    model.add(Dense(512, activation="relu"))
    model.add(Dropout(0.1))
    model.add(Dense(64, activation="relu"))
    model.add(Dense(1, activation="sigmoid"))
    
    return model

def main():
    x_train, y_train = data_generator(TRAIN_CSV)
    x_val, y_val = data_generator(VALIDATION_CSV)
    
    input_shape = x_train.shape[1:]
# #     y_train = pd.DataFrame(data=y_train)
# #     y_val = pd.DataFrame(data=y_val)
#     print(y_val)
#     model = create_model(input_shape)
#     model.summary()
    
#     optimizer = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
#     model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=["accuracy"])
    
#     checkpoint = ModelCheckpoint("model-{val_dice:.2f}.h5", monitor="val_dice", verbose=1, save_best_only=True,
#                                  save_weights_only=True, mode="max")
#     stop = EarlyStopping(monitor="val_dice", patience=PATIENCE, mode="max")
#     reduce_lr = ReduceLROnPlateau(monitor="val_dice", factor=0.2, patience=5, min_lr=1e-6, verbose=1, mode="max")
    
# #     aug = ImageDataGenerator(rotation_range=10, zoom_range=0.1,
# #                          width_shift_range=0.2, height_shift_range=0.2, shear_range=0.1,
# #                          horizontal_flip=True, fill_mode="nearest")
    
# #     history = model.fit_generator(aug.flow(x_train, y_train, batch_size=BATCH_SIZE),
# #                                   validation_data=(x_val, y_val), 
# #                                   callbacks=[checkpoint, reduce_lr, stop],
# #                                   steps_per_epoch=len(x_train) // BATCH_SIZE,
# #                                   epochs=EPOCHS)

#     model.fit(x_train, y_train,
#               batch_size=BATCH_SIZE,
#               epochs=EPOCHS,
#               validation_data=(x_val, y_val),
#               shuffle=True,
#               verbose=1)


In [10]:
main()